In [1]:
from torch import Tensor
from torch import nn
import logging as log
from typing import Optional
from PIL import Image
import torchvision.transforms.functional as TTF
import torch
import numpy as np

import warnings
warnings.filterwarnings('ignore')

In [2]:
class Flatten(nn.Module):
    "Flatten `x` to a single dimension, often used at the end of a model. `full` for rank-1 tensor"
    def __init__(self, full:bool=False):
        super().__init__()
        self.full = full

    def forward(self, x):
        return x.view(-1) if self.full else x.view(x.size(0), -1)

class AdaptiveConcatPool2d(nn.Module):
    "Layer that concats `AdaptiveAvgPool2d` and `AdaptiveMaxPool2d`." # from pytorch
    def __init__(self, sz:Optional[int]=None): 
        "Output will be 2*sz or 2 if sz is None"
        super().__init__()
        self.output_size = sz or 1
        self.ap = nn.AdaptiveAvgPool2d(self.output_size)
        self.mp = nn.AdaptiveMaxPool2d(self.output_size)
    def forward(self, x): return torch.cat([self.mp(x), self.ap(x)], 1)
    
def myhead(nf, nc):
    return \
    nn.Sequential(        # the dropout is needed otherwise you cannot load the weights
            AdaptiveConcatPool2d(),
            Flatten(),
            nn.BatchNorm1d(nf),
            nn.Dropout(p=0.25),
            nn.Linear(nf, 512),
            nn.ReLU(True),
            nn.BatchNorm1d(512),
            nn.Dropout(p=0.5),
            nn.Linear(512, nc),
        )

In [ ]:
import torch
import torchvision
# for densenet201
my_model=torchvision.models.densenet201() 

modules=list(my_model.children())
modules.pop(-1) 
# modules.pop(-1)
temp=nn.Sequential(nn.Sequential(*modules))
tempchildren=list(temp.children())
# nc is no. of classes and and nf is no, of features(here nf will vary according to architecture like for densenet201
# it is 3840), so to determine the value of nf just run this cell you will get a size mismatch error and then just replace
# with the size from the error
tempchildren.append(myhead(nf, nc))

my_d201=nn.Sequential(*tempchildren)

In [ ]:
# now load the model which you should have saved in classifier using "torch.save"
model=my_r101
weighties = torch.load('./name .pth')
model.load_state_dict(weighties['state_dict'])

In [ ]:
# GET THE PREDICTIONS
img = path_of_image
softmaxer = torch.nn.Softmax(dim=1)
model.eval()
image = Image.open(img)
image = image.resize((448, 448))
x = TTF.to_tensor(image)
x = x.div_(255)
x.unsqueeze_(0)
print(x.shape)
raw_out = model(x)
out = softmaxer(raw_out)
print(out[0])
